## Feautre selection done by testing unsupervised and supervised methods
Feature selection is doen after explorative data analysis. 
If potential groups of damage-influencing variables are clear than supervised techniques will be used, if underlying groups of variables are unknown than PCA (unsupervised method) is used.



In [ ]:

import numpy as np
import pandas as pd


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


## define explanatory variables and target variables and Standardize

In [ ]:
## select only numeric columns, exclude als .specfic-columns or with string or datetime
df_p125_t = df_p125.select_dtypes(include = ['int64'], exclude=["object"])  
df_p125_t
## determine only important variables

y_direct = df_p125['Target_direct'] #  target var for direct cost due to damages on buiness contents 
y_indirect = df_p125['Target_indirect'] # target var for indirect costs due to buiness interuptions 
X = df_p125.loc[ :, df_p125.columns != ['Target_direct', 'Target_indirect'] ]
X = X.iloc[:,:] # assign column 9 to 21 as x variable - the features

X = StandardScaler().fit_transform(X) # standarize the variables
#df['Attrition_Flag'].replace('Existing Customer','1',inplace=True)  # categorize string columns
#df['Attrition_Flag'].replace('Attrited Customer','0',inplace=True)


## PCA

*Example from: https://towardsdatascience.com/principal-component-analysis-with-python-an-example-for-beginners-by-a-beginner-ac052eff45c*


#### PCA on direct costs

In [ ]:

## We will start by using only the first 2 leading principal components, and then explore 3 principal components and 4 principal components.
pca=PCA(n_components=2)
PC=pca.fit_transform(X)
principalDF=pd.DataFrame(data=PC, columns=['pc1','pc2'])
df_pca = pd.concat([principalDF, df_p125[['Target_direct']]], axis = 1)
df_pca.head()
X.columns.tolist()
#components #=components[9:21]

PCloadings = pca.components_.T * np.sqrt(pca.explained_variance_)
components = df_p125.loc[:, df_p125.columns != 'Target'].columns.tolist()  # use X before standarized

loadingdf = pd.DataFrame(PCloadings,columns=('PC1','PC2'))
loadingdf["variable"] = components
loadingdf
### Visual selection of important variables by weights of PCs
fig = ex.scatter(x=loadingdf['PC1'],y=loadingdf['PC2'],text=loadingdf['variable'],)
fig.update_layout(
height = 600, width = 500,
title_text='loadings plot')
fig.update_traces(textposition='bottom center')
fig.add_shape(type="line",
    x0 = -0, y0 = -0.5, x1 = -0, y1 = 2.5,
    line = dict(color="RoyalBlue",width=3)
)
fig.add_shape(type="line",
    x0 = -1, y0 = 0, x1 = 1, y1 = 0,
    line=dict(color="RoyalBlue",width=3)
)
fig.show()

#### PCA on indirect costs

## Discretized continous variabels: 
E.g. by an equal frequency discretization
use coarse discrete classes for each varible e.g. 5 classes used in Can ho City-DS